In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [84]:
train = pd.read_csv('./data/house_train.csv')
test = pd.read_csv('./data/house_test.csv')

In [85]:
train

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15030,15030,20141014T000000,610685.0,4,2.50,2520,6023,2.0,0,0,...,9,2520,0,2014,0,98056,47.5137,-122.167,2520,6023
15031,15031,20150326T000000,1007500.0,4,3.50,3510,7200,2.0,0,0,...,9,2600,910,2009,0,98136,47.5537,-122.398,2050,6200
15032,15032,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
15033,15033,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200


In [86]:
train['bedrooms'].unique()

array([ 3,  2,  5,  4,  6,  1,  7,  9,  8,  0, 10], dtype=int64)

In [87]:
train['waterfront'].unique()

array([0, 1], dtype=int64)

waterfront가 분류 문제로 풀기에 적합한 것 같다.

In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15035 entries, 0 to 15034
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15035 non-null  int64  
 1   date           15035 non-null  object 
 2   price          15035 non-null  float64
 3   bedrooms       15035 non-null  int64  
 4   bathrooms      15035 non-null  float64
 5   sqft_living    15035 non-null  int64  
 6   sqft_lot       15035 non-null  int64  
 7   floors         15035 non-null  float64
 8   waterfront     15035 non-null  int64  
 9   view           15035 non-null  int64  
 10  condition      15035 non-null  int64  
 11  grade          15035 non-null  int64  
 12  sqft_above     15035 non-null  int64  
 13  sqft_basement  15035 non-null  int64  
 14  yr_built       15035 non-null  int64  
 15  yr_renovated   15035 non-null  int64  
 16  zipcode        15035 non-null  int64  
 17  lat            15035 non-null  float64
 18  long  

In [89]:
test

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,15208,20140512T000000,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
1,15209,20150312T000000,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570
2,15210,20150124T000000,4,3.00,2950,5000,2.0,0,3,3,9,1980,970,1979,0,98126,47.5714,-122.375,2140,4000
3,15211,20141205T000000,2,1.00,1200,9850,1.0,0,0,4,7,1200,0,1921,0,98002,47.3089,-122.210,1060,5095
4,15212,20140514T000000,4,1.75,1620,4980,1.0,0,0,4,7,860,760,1947,0,98133,47.7025,-122.341,1400,4980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6550,21758,20141002T000000,4,2.75,2770,3852,2.0,0,0,3,8,2770,0,2014,0,98178,47.5001,-122.232,1810,5641
6551,21759,20150416T000000,3,2.25,1190,1200,3.0,0,0,3,8,1190,0,2008,0,98103,47.6542,-122.346,1180,1224
6552,21760,20141015T000000,5,3.75,4470,8088,2.0,0,0,3,11,4470,0,2008,0,98004,47.6321,-122.200,2780,8964
6553,21761,20150126T000000,3,2.00,1490,1126,3.0,0,0,3,8,1490,0,2014,0,98144,47.5699,-122.288,1400,1230


In [90]:
X = train.loc[:, ['bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'view']]
X_train, X_test, y_train, y_test = train_test_split(X, train['waterfront'])

정보를 몇 가지만 넣어서 해보자.

In [91]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11276, 5), (3759, 5), (11276,), (3759,))

In [92]:
X_train

,bathrooms,sqft_living,sqft_lot,floors,view
13434,1.00,1080,4000,1.0,0
10074,2.50,1820,8217,1.0,0
7786,1.75,1020,5000,1.0,0
873,1.75,2490,7834,1.0,3
11060,1.00,1340,7920,1.0,0
...,...,...,...,...,...
5095,2.25,1530,6600,1.0,0
14913,3.50,2630,3757,2.0,0
921,1.00,1250,17380,1.0,0
5287,1.00,870,5750,1.0,0


In [93]:
X_test

,bathrooms,sqft_living,sqft_lot,floors,view
14161,2.50,2009,4800,2.0,0
12749,2.50,2480,176418,1.5,3
10642,1.75,2250,8580,1.0,0
12208,2.50,3220,7873,2.0,0
965,1.75,1290,3870,1.0,0
...,...,...,...,...,...
9328,1.00,720,5600,1.0,0
13741,4.00,5610,11063,3.0,0
6099,2.25,2130,11900,2.0,0
8553,2.50,2460,4774,2.0,0


In [94]:
y_train

13434    0
10074    0
7786     0
873      0
11060    0
        ..
5095     0
14913    0
921      0
5287     0
12251    0
Name: waterfront, Length: 11276, dtype: int64

In [95]:
y_test

14161    0
12749    0
10642    0
12208    0
965      0
        ..
9328     0
13741    0
6099     0
8553     0
2826     0
Name: waterfront, Length: 3759, dtype: int64

In [96]:
knn = KNeighborsClassifier(n_neighbors=3)

In [97]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [98]:
scores = cross_val_score(knn, X, train['waterfront'])
print("교차 검증 점수 : {}".format(scores))

교차 검증 점수 : [0.9930163  0.99235118 0.99268374 0.99235118 0.99201862]


In [99]:
X = train.drop(columns=['date', 'waterfront'])
X_train, X_test, y_train, y_test = train_test_split(X, train['waterfront'])

In [100]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11276, 19), (3759, 19), (11276,), (3759,))

In [101]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
scores = cross_val_score(knn, X, train['waterfront'])
print("교차 검증 점수 : {}".format(scores))

교차 검증 점수 : [0.99268374 0.99168607 0.99102095 0.99135351 0.99068839]


In [102]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3)
print("교차 검증 점수 : \n{}".format(cross_val_score(knn, 
                                               X, 
                                               train['waterfront'], cv=kfold)))

교차 검증 점수 : 
[0.99122107 0.99162011 0.99161844]


In [103]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("교차 검증 점수 : \n{}".format(cross_val_score(knn, 
                                               X, 
                                               train['waterfront'], cv=kfold)))

교차 검증 점수 : 
[0.99261772 0.99181963 0.99002195]


In [104]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
print("교차 검증 점수 : \n{}".format(cross_val_score(knn, 
                                               X, 
                                               train['waterfront'], cv=kfold)))

교차 검증 점수 : 
[0.99168607 0.99135351 0.99168607 0.98969072 0.99035584]
